In [112]:
#imports

#google sheets
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials
import time

#data manipulation
import pandas as pd
import numpy as np
import warnings
import locale
import datetime

locale.setlocale(locale.LC_TIME, 'es_MX.UTF-8')
warnings.filterwarnings("ignore")

In [55]:
scope = [
   'https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

google_key_file = 'service_key.json'

credentials = ServiceAccountCredentials.from_json_keyfile_name(google_key_file, scope)
gc = gspread.authorize(credentials)

---

In [252]:
def GetDataFromGS(sheet):
    rows =  sheet.get_all_values()
    df = pd.DataFrame.from_records(rows[1:])
    df.columns = [column.strip() for column in rows[0]]
    return df

In [253]:
sh = gc.open('Ventas Enero 2021')
worksheet_list = sh.worksheets()

ventas = pd.DataFrame()
for sheet in worksheet_list:
    time.sleep(2)
    venta = GetDataFromGS(sheet)
    ventas = ventas.append(venta)
display(ventas)

,fecha,cliente,cantidad,producto,precio_u,importe
0,2/1/2021,Salinas,"30,8",pechuga,70,"$ 2.156,00"
1,2/1/2021,Salinas,"20,4",pierna con muslo,43,"$ 877,20"
2,2/1/2021,Salinas,"10,5",retazo,35,"$ 367,50"
3,2/1/2021,Jesus Aguinaga,"7,3",pierna con muslo,45,"$ 328,50"
4,2/1/2021,Jesus Aguinaga,"4,1",pechuga,80,"$ 328,00"
...,...,...,...,...,...,...
18,31/1/2021,La Suiza,"3,6",retazo,35,"$ 126,00"
19,31/1/2021,El Tacote,"4,2",pechuga,68,"$ 285,60"
20,31/1/2021,El Tacote,"2,5",molleja,35,"$ 87,50"
21,31/1/2021,El Tacote,"4,2",pulpa,95,"$ 399,00"


In [254]:
def CleanData(data: pd.DataFrame):
    df = data.copy()
    df['fecha'] = pd.to_datetime(df['fecha'], format="%d/%m/%Y").dt.strftime('%d/%m/%Y')
    
    df['cantidad'] = pd.to_numeric(df['cantidad'].str.replace(',','.'))
    
    df['precio_u'] = df['precio_u'].str.replace(',','.')
    df['precio_u'] = pd.to_numeric(df['precio_u'].str.replace('$',''))
    
    df['importe'] = df['importe'].str.replace('.','')
    df['importe'] = df['importe'].str.replace(',','.')
    df['importe'] = pd.to_numeric(df['importe'].str.replace('$',''))
    
    return df

In [257]:
a = CleanData(ventas)

display(a)

,fecha,cliente,cantidad,producto,precio_u,importe
0,02/01/2021,Salinas,30.8,pechuga,70.0,2156.0
1,02/01/2021,Salinas,20.4,pierna con muslo,43.0,877.2
2,02/01/2021,Salinas,10.5,retazo,35.0,367.5
3,02/01/2021,Jesus Aguinaga,7.3,pierna con muslo,45.0,328.5
4,02/01/2021,Jesus Aguinaga,4.1,pechuga,80.0,328.0
...,...,...,...,...,...,...
18,31/01/2021,La Suiza,3.6,retazo,35.0,126.0
19,31/01/2021,El Tacote,4.2,pechuga,68.0,285.6
20,31/01/2021,El Tacote,2.5,molleja,35.0,87.5
21,31/01/2021,El Tacote,4.2,pulpa,95.0,399.0
